In [1]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [ ]:
print(newsgroups_train.target_names)
print(newsgroups_train.target[0]) 

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
7
[7 4 4 ... 3 1 8]


## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [27]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


In [61]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
import random

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

### **1)** Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [76]:
def analizar_similitud_documentos(X_train, y_train, target_names, vectorizer, n=5, m=10, seed=42):
    import random
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity

    random.seed(seed)
    idxs = random.sample(range(0, X_train.shape[0]), n)
    vocab = np.array(vectorizer.get_feature_names_out())

    for i in idxs:
        print("DOC ELEGIDO:")
        print(f"{target_names[y_train[i]]} -->", end=" ")

        fila = X_train[i].toarray().flatten()
        top_words_idx = np.argsort(fila)[::-1][:m]
        print(", ".join(vocab[top_words_idx]))

        cossim = cosine_similarity(X_train[i], X_train)[0]
        mostsim = np.argsort(cossim)[::-1][1:n+1]

        print("DOCS MÁS SIMILARES:")
        for j in mostsim:
            fila_j = X_train[j].toarray().flatten()
            top_words_j = np.argsort(fila_j)[::-1][:m]
            simil = cossim[j]
            print(f"{target_names[y_train[j]]} (simil: {simil:.3f}) -->", ", ".join(vocab[top_words_j]))
        print("-------")



Se aplica tanto **TF-IDF** como **CountVectorizer** y se utiliza la similaridad coseno para comparar documentos. Se seleccionan documentos al azar y se identifican los 5 más similares en función de la similaridad coseno. En cada caso se imprime la clase del documento elegido, la clase de los más similares, el valor de similaridad coseno y los 10 términos con mayor peso según el vectorizador utilizado (**TF-IDF** o **CountVectorizer**).

**Considerando TF-IDF:**

Se observa que en algunas clases, incluso con valores bajos de similaridad coseno, los documentos más similares pertenecen a la misma clase. En otros casos ocurre lo contrario: se obtienen valores de similaridad más altos, pero los documentos similares pertenecen a clases distintas. Esto sugiere que la efectividad de la vectorización depende del tipo de contenido de cada documento y de cada clase.

En varios casos, los documentos más similares comparten términos con alto peso TF-IDF. Por ejemplo, el término `pov` aparece entre los más relevantes tanto en el documento elegido de la clase `comp.graphics` como en los documentos más similares, que también pertenecen a esa misma clase. Lo mismo se observa en la clase `comp.sys.mac.hardware`, donde los documentos cercanos comparten términos representativos como `processor` y coinciden en la misma categoría.

También se observa que, en algunos casos, los documentos más similares no pertenecen a la misma clase, pero comparten vocabulario temáticamente relacionado. Por ejemplo, el documento elegido que pertenece la clase `rec.autos` resultó tener como uno de sus documentos más similares a otro de la clase `rec.motorcycles`. Aunque pertenecen a categorías distintas, comparten términos específicos como `oil`, lo cual explica su cercanía en el espacio vectorial.

Esto sugiere que los términos con mayor peso en el vector TF-IDF son clave para definir la dirección del vector utilizado en el cálculo del coseno. Algunos de estos términos más representativos coinciden entre documentos de la misma clase, lo que refuerza la coherencia temática. Sin embargo, en otros casos también aparecen coincidencias entre clases diferentes, lo que refleja solapamientos en el vocabulario y puede generar ambigüedades en la comparación.

**Considerando CountVectorizer:** 

En este caso, los valores de similaridad coseno son más altos en general, pero los documentos más similares no suelen pertenecer a la misma clase que el documento elegido, ni coinciden entre ellos. Esto se debe a que la vectorización asigna pesos (valores numéricos) altos a términos comunes entre todas las clases, que no aportan información útil para distinguir temáticas (por ejemplo, palabras como `the`, `and`, `to`, etc.). Esto reduce la capacidad del modelo para discriminar documentos según su contenido real.

Este problema se corrige al utilizar TF-IDF, donde el término IDF penaliza las palabras frecuentes en todos los documentos. De esa forma, se reduce el peso de términos que no son representativos de una clase en particular.



In [77]:
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
target_names = newsgroups_train.target_names
analizar_similitud_documentos(X_train, y_train, target_names, tfidfvect)


DOC ELEGIDO:
rec.sport.hockey --> series, coverage, canada, watch, games, swamped, news, precedence, televised, nationally
DOCS MÁS SIMILARES:
rec.sport.hockey (simil: 0.225) --> detroit, chicago, games, exciting, classic, mon, playoff, boston, fan, realize
talk.politics.mideast (simil: 0.217) --> the, and, they, were, was, that, he, to, you, it
sci.crypt (simil: 0.216) --> the, to, it, not, is, rights, this, be, government, clipper
alt.atheism (simil: 0.213) --> the, of, is, god, that, in, to, atheists, universe, it
sci.crypt (simil: 0.211) --> the, internet, anonymity, anonymous, and, privacy, of, is, to, are
-------
DOC ELEGIDO:
comp.sys.mac.hardware --> processor, 040, transidental, speed, its, is, functions, 486, increase, can
DOCS MÁS SIMILARES:
comp.sys.mac.hardware (simil: 0.354) --> speed, 20ns, memory, processor, clock, system, as, the, kitabjian, dhk
comp.sys.mac.hardware (simil: 0.313) --> dock, processor, 040, apple, notebook, duo, powerbook, power, the, okay
comp.sys.mac.

In [78]:
countvect = CountVectorizer()
X_train = countvect.fit_transform(newsgroups_train.data)
analizar_similitud_documentos(X_train, y_train, target_names, countvect)

DOC ELEGIDO:
rec.sport.hockey --> the, is, in, not, and, to, series, are, how, games
DOCS MÁS SIMILARES:
talk.politics.mideast (simil: 0.644) --> the, to, not, is, in, of, it, and, that, they
sci.crypt (simil: 0.643) --> the, and, to, of, is, in, are, on, or, that
rec.autos (simil: 0.639) --> the, of, is, in, and, to, are, they, that, radar
sci.med (simil: 0.638) --> the, and, of, in, is, to, it, candida, this, for
comp.windows.x (simil: 0.637) --> the, to, is, and, of, in, for, it, this, widget
-------
DOC ELEGIDO:
comp.sys.mac.hardware --> is, processor, to, its, can, you, not, speed, the, or
DOCS MÁS SIMILARES:
alt.atheism (simil: 0.609) --> of, is, the, to, that, and, it, not, or, can
soc.religion.christian (simil: 0.598) --> to, is, the, it, that, but, not, you, have, be
talk.politics.guns (simil: 0.590) --> the, is, to, of, that, it, in, and, on, for
alt.atheism (simil: 0.589) --> to, the, is, that, of, you, it, are, and, in
soc.religion.christian (simil: 0.581) --> the, that, is

### **2)** Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

### Análisis de sensibilidad de parámetros

Se prueban distintas combinaciones de parámetros del vectorizador y del modelo para evaluar cómo afectan al F1-score macro en el conjunto de test.

**Parámetros del vectorizador:**

- `stop_words`: se usa `'english'` para eliminar palabras muy comunes que no ayudan a distinguir clases. También se prueba `None`.
- `max_features`: para limitar la cantidad de palabras del vocabulario (uso 1000, 10000 y 100000).
- `ngram_range`: pruebo unigrams `(1,1)` y unigrams + bigrams `(1,2)` para capturar contexto.
- `min_df` y `max_df`: para filtrar palabras que aparecen en muy pocos o en casi todos los documentos.

**Parámetros del modelo:**

- `alpha`: con esto se controla el suavizado. Pruebo los valores 0.01, 0.1 1.0 y 10.0.
- `model_class`: comparo `MultinomialNB` y `ComplementNB`.

Evalúo el impacto de cada parámetro usando F1-score para ver qué combinaciones dan mejores resultados.


In [97]:
def evaluar_naive_bayes_parametrico(
    vectorizer_class,
    vectorizer_params,
    model_class,
    alpha,
    train_data,
    test_data,
    train_labels,
    test_labels
):
    # Instanciación del vectorizador con parámetros
    vectorizer = vectorizer_class(**vectorizer_params)
    X_train = vectorizer.fit_transform(train_data)
    X_test = vectorizer.transform(test_data)

    # Instanciación y entrenamiento del modelo
    modelo = model_class(alpha=alpha)
    modelo.fit(X_train, train_labels)
    y_pred = modelo.predict(X_test)

    # Evaluación
    f1 = f1_score(test_labels, y_pred, average='macro')
    print(f"{model_class.__name__} con {vectorizer_class.__name__} | "
          f"alpha={alpha}, params={vectorizer_params} -> F1-Score: {f1:.4f}")

In [99]:
# Valores base para cada parámetro
base_params = {
    'stop_words': 'english',
    'max_features': 10000,
    'ngram_range': (1, 1),
    'min_df': 2,
    'max_df': 0.9
}
base_alpha = 1.0
vectorizer_classes = [TfidfVectorizer, CountVectorizer]
model_classes = [MultinomialNB, ComplementNB]

# Variación 1: cambiar alpha
print("### Variación de alpha")
for alpha in [0.01, 0.1, 1.0, 10.0]:
    evaluar_naive_bayes_parametrico(
        vectorizer_class=TfidfVectorizer,
        vectorizer_params=base_params,
        model_class=MultinomialNB,
        alpha=alpha,
        train_data=newsgroups_train.data,
        test_data=newsgroups_test.data,
        train_labels=newsgroups_train.target,
        test_labels=newsgroups_test.target
    )

# Variación 2: cambiar stop_words
print("\n### Variación de stop_words")
for sw in ['english', None]:
    params = base_params.copy()
    params['stop_words'] = sw
    evaluar_naive_bayes_parametrico(
        vectorizer_class=TfidfVectorizer,
        vectorizer_params=params,
        model_class=MultinomialNB,
        alpha=base_alpha,
        train_data=newsgroups_train.data,
        test_data=newsgroups_test.data,
        train_labels=newsgroups_train.target,
        test_labels=newsgroups_test.target
    )

# Variación 3: cambiar ngram_range
print("\n### Variación de ngram_range")
for ngram in [(1, 1), (1, 2)]:
    params = base_params.copy()
    params['ngram_range'] = ngram
    evaluar_naive_bayes_parametrico(
        vectorizer_class=TfidfVectorizer,
        vectorizer_params=params,
        model_class=MultinomialNB,
        alpha=base_alpha,
        train_data=newsgroups_train.data,
        test_data=newsgroups_test.data,
        train_labels=newsgroups_train.target,
        test_labels=newsgroups_test.target
    )

# Variación 4: cambiar max_features
print("\n### Variación de max_features")
for max_feat in [1000, 10000, 100000]:
    params = base_params.copy()
    params['max_features'] = max_feat
    evaluar_naive_bayes_parametrico(
        vectorizer_class=TfidfVectorizer,
        vectorizer_params=params,
        model_class=MultinomialNB,
        alpha=base_alpha,
        train_data=newsgroups_train.data,
        test_data=newsgroups_test.data,
        train_labels=newsgroups_train.target,
        test_labels=newsgroups_test.target
    )
    
# Variación 5: cambiar modelo
print("\n### Variación de model_class")
for model_class in [MultinomialNB, ComplementNB]:
    evaluar_naive_bayes_parametrico(
        vectorizer_class=TfidfVectorizer,
        vectorizer_params=base_params,
        model_class=model_class,
        alpha=base_alpha,
        train_data=newsgroups_train.data,
        test_data=newsgroups_test.data,
        train_labels=newsgroups_train.target,
        test_labels=newsgroups_test.target
    )

# Variación 6: cambiar vectorizador
print("\n### Variación de vectorizer_class")
for vect_class in [TfidfVectorizer, CountVectorizer]:
    evaluar_naive_bayes_parametrico(
        vectorizer_class=vect_class,
        vectorizer_params=base_params,
        model_class=MultinomialNB,
        alpha=base_alpha,
        train_data=newsgroups_train.data,
        test_data=newsgroups_test.data,
        train_labels=newsgroups_train.target,
        test_labels=newsgroups_test.target
    )

# Variación 7: cambiar min_df y max_df
print("\n### Variación de min_df y max_df")
min_max_df_combos = [
    (1, 1.0),
    (2, 0.9),
    (5, 0.8)
]
for min_df, max_df in min_max_df_combos:
    params = base_params.copy()
    params['min_df'] = min_df
    params['max_df'] = max_df
    evaluar_naive_bayes_parametrico(
        vectorizer_class=TfidfVectorizer,
        vectorizer_params=params,
        model_class=MultinomialNB,
        alpha=base_alpha,
        train_data=newsgroups_train.data,
        test_data=newsgroups_test.data,
        train_labels=newsgroups_train.target,
        test_labels=newsgroups_test.target
    )

### Variación de alpha
MultinomialNB con TfidfVectorizer | alpha=0.01, params={'stop_words': 'english', 'max_features': 10000, 'ngram_range': (1, 1), 'min_df': 2, 'max_df': 0.9} -> F1-Score: 0.6518
MultinomialNB con TfidfVectorizer | alpha=0.1, params={'stop_words': 'english', 'max_features': 10000, 'ngram_range': (1, 1), 'min_df': 2, 'max_df': 0.9} -> F1-Score: 0.6634
MultinomialNB con TfidfVectorizer | alpha=1.0, params={'stop_words': 'english', 'max_features': 10000, 'ngram_range': (1, 1), 'min_df': 2, 'max_df': 0.9} -> F1-Score: 0.6469
MultinomialNB con TfidfVectorizer | alpha=10.0, params={'stop_words': 'english', 'max_features': 10000, 'ngram_range': (1, 1), 'min_df': 2, 'max_df': 0.9} -> F1-Score: 0.5853

### Variación de stop_words
MultinomialNB con TfidfVectorizer | alpha=1.0, params={'stop_words': 'english', 'max_features': 10000, 'ngram_range': (1, 1), 'min_df': 2, 'max_df': 0.9} -> F1-Score: 0.6469
MultinomialNB con TfidfVectorizer | alpha=1.0, params={'stop_words': None, '

### Mejores resultados por parámetro

- Mejor `alpha`: 0.1, F1-Score: 0.6634  
- Mejor `stop_words`: 'english', F1-Score: 0.6469  
- Mejor `ngram_range`: (1, 1), F1-Score: 0.6469  
- Mejor `max_features`: 100000, F1-Score: 0.6512  
- Mejor `model_class`: ComplementNB, F1-Score: 0.6664  
- Mejor `vectorizer_class`: TfidfVectorizer, F1-Score: 0.6469  
- Mejor combinación `min_df` y `max_df`: 5 y 0.8, F1-Score: 0.6487  

In [101]:
# Pruebo con la mejor combinación de parámetros encontrada
print("\n### Evaluación con la mejor combinación de parámetros")

mejores_params_vect = {
    'stop_words': 'english',
    'max_features': 100000,
    'ngram_range': (1, 1),
    'min_df': 5,
    'max_df': 0.8
}

evaluar_naive_bayes_parametrico(
    vectorizer_class=TfidfVectorizer,
    vectorizer_params=mejores_params_vect,
    model_class=ComplementNB,
    alpha=0.1,
    train_data=newsgroups_train.data,
    test_data=newsgroups_test.data,
    train_labels=newsgroups_train.target,
    test_labels=newsgroups_test.target
)



### Evaluación con la mejor combinación de parámetros
ComplementNB con TfidfVectorizer | alpha=0.1, params={'stop_words': 'english', 'max_features': 100000, 'ngram_range': (1, 1), 'min_df': 5, 'max_df': 0.8} -> F1-Score: 0.6745


Usando la mejor combinación de parámetros encontrada en el análisis de sensibilidad, se obtiene el mejor resultado de todos: F1-Score de 0.6745.

### **3)** Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

In [108]:
vectorizer = TfidfVectorizer(**mejores_params_vect)
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_words = X_train.T
vocabulario = vectorizer.get_feature_names_out()
word_to_idx = {word: i for i, word in enumerate(vocabulario)}
palabras_utiles = [w for w in vocabulario if w.isalpha() and len(w) > 3]
print(palabras_utiles[:10000])


['aaron', 'abandon', 'abandoned', 'abbey', 'abbott', 'abbreviation', 'abdominal', 'abdullah', 'abide', 'abiding', 'abilities', 'ability', 'able', 'abnormal', 'aboard', 'abode', 'abolish', 'abolished', 'abomination', 'abominations', 'abort', 'aborted', 'abortion', 'abound', 'abraham', 'abridged', 'abroad', 'abruptly', 'absence', 'absent', 'absolute', 'absolutely', 'absolutes', 'absolutist', 'absorb', 'absorbed', 'absorbing', 'absorption', 'abstinence', 'abstract', 'abstraction', 'abstracts', 'absurd', 'absurdity', 'abundance', 'abundant', 'abuse', 'abused', 'abuses', 'abusing', 'abusive', 'abyss', 'academia', 'academic', 'academics', 'academy', 'accelerate', 'accelerated', 'accelerating', 'acceleration', 'accelerator', 'accelerators', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accessable', 'accessed', 'accesses', 'accessible', 'accessing', 'accessories', 'accessory', 'accident', 'accidental', 'accidentally', 'accidently', 'accidents', 'accommoda

In [110]:
palabras = ['abolished', 'creature', 'engineering', 'fascist', 'prosecutors']

def analizar_similitud_palabras(X_words, vocabulario, word_to_idx, palabras, top_n=5):
    for palabra in palabras:
        if palabra not in word_to_idx:
            print(f"{palabra} no encontrada en el vocabulario.")
            continue

        idx = word_to_idx[palabra]
        vec = X_words[idx]
        similitudes = cosine_similarity(vec, X_words)[0]
        similares_idx = np.argsort(similitudes)[::-1][1:top_n+1]

        print(f"PALABRA ELEGIDA: {palabra}")
        print("PALABRAS MÁS SIMILARES:")
        for i in similares_idx:
            print(f"{vocabulario[i]} (simil: {similitudes[i]:.3f})")
        print("-------")
        
analizar_similitud_palabras(X_words, vocabulario, word_to_idx, palabras)

PALABRA ELEGIDA: abolished
PALABRAS MÁS SIMILARES:
forfeiture (simil: 0.519)
axe (simil: 0.428)
selective (simil: 0.400)
irs (simil: 0.394)
insult (simil: 0.314)
-------
PALABRA ELEGIDA: creature
PALABRAS MÁS SIMILARES:
strategic (simil: 0.332)
coordinated (simil: 0.317)
insulted (simil: 0.286)
neill (simil: 0.257)
destiny (simil: 0.227)
-------
PALABRA ELEGIDA: engineering
PALABRAS MÁS SIMILARES:
ng (simil: 0.274)
engineers (simil: 0.175)
wong (simil: 0.161)
korea (simil: 0.159)
electrical (simil: 0.154)
-------
PALABRA ELEGIDA: fascist
PALABRAS MÁS SIMILARES:
fascism (simil: 0.645)
practiced (simil: 0.428)
discrimination (simil: 0.347)
ending (simil: 0.326)
soviet (simil: 0.318)
-------
PALABRA ELEGIDA: prosecutors
PALABRAS MÁS SIMILARES:
bullock (simil: 0.561)
gerard (simil: 0.393)
adl (simil: 0.341)
francisco (simil: 0.315)
documenting (simil: 0.306)
-------
